In [2]:
import numba
from numba import cuda
"""
Implementación de scan para Numba CUDA.
Basado en https://developer.nvidia.com/gpugems/GPUGems3/gpugems3_ch39.html
"""

def scan(x, out, MAX_TPB=128):
  
    n = x.size
    tpb = MAX_TPB 
    # Elementos por bloque
    epb = tpb * 2 
    # Número de bloques completos
    blocks = n // epb
    # Elementos en el último bloque
    elb = n % epb

    # Si sólo tenemos un bloque
    if blocks == 0 or (blocks == 1 and elb == 0):
        # Mínima potencia de 2 que incluye todos los elementos
        exponente2 = np.int(np.ceil(np.log2(elb)))
        # Total de elementos en el último bloque, teniendo en cuenta ceros extra de relleno
        total_elb = 2 ** exponente2
        # Total de hebras por bloque
        my_tpb = total_elb // 2
        # Memoria compartida
        sm_size = total_elb * x.dtype.itemsize
        
        aux = cuda.device_array(1, x.dtype)
        not_full_block_scan[1, my_tpb, 0, sm_size](x, out, aux, 0, elb, 0)
        
        return aux

    else:
        n_scans = blocks if elb == 0 else blocks + 1
        aux = cuda.device_array(n_scans, x.dtype)

        # Memoria compartida
        sm_size = epb * x.dtype.itemsize

        # Prescan de todos los bloques
        prescan[blocks, tpb, 0, sm_size](x, out, aux)

        # Prescan del último bloque, si procede
        if elb != 0:
            # Mínima potencia de 2 que incluye todos los elementos
            exponente2 = np.int(np.ceil(np.log2(elb)))
            # Total de elementos en el último bloque, teniendo en cuenta ceros extra de relleno
            total_elb = 2 ** exponente2
            # Total de hebras por bloque
            my_tpb = total_elb // 2
            # Memoria compartida
            sm_size = total_elb * x.dtype.itemsize
            
            not_full_block_scan[1, my_tpb, 0, sm_size](x, out, aux, n_scans - 1, elb, n - elb)

        d_out2 = cuda.device_array(aux.shape, aux.dtype)
        total = scan(aux, d_out2, MAX_TPB)

        scan_sum[n_scans, tpb](out, d_out2)

        return total

@cuda.jit
def scan_sum(data, aux):
    temp = cuda.shared.array(1, numba.int32)

    # Índices
    bidx = cuda.blockIdx.x 
    tidx = cuda.threadIdx.x
    eidx = cuda.grid(1) * 2
    
    # Si estamos en la primera hebra guardamos el
    # auxiliar lo acumulado del bloque anteior
    if tidx == 0:
        temp[0] = aux[bidx]

    cuda.syncthreads()
    
    # Sumamos lo acumulado del bloque anterior
    if eidx <= data.size:
        data[eidx] += aux[bidx] 

        if eidx + 1 < data.size:
            data[eidx + 1] += aux[bidx]

@cuda.jit
def prescan(data_in, data_out, aux):
    
    # Índices y memoria compartida
    shared_aux = cuda.shared.array(0, numba.int32)
    tidx = cuda.threadIdx.x 
    idx = cuda.grid(1) 
    bidx = cuda.blockIdx.x 
    blockSize = cuda.blockDim.x
    
    # Cargamos datos en memoria compartida
    shared_aux[2 * tidx] = data_in[2 * idx]
    shared_aux[2 * tidx + 1] = data_in[2 * idx + 1]
    
    offset = 1

    # Construcción del árbol up-sweepe
    d = blockSize
    while d > 0:
        cuda.syncthreads()
        
        if tidx < d:
            ai = offset * (2 * tidx + 1) - 1
            bi = offset * (2 * tidx + 2) - 1

            shared_aux[bi] += shared_aux[ai]
        offset <<= 1 
        d >>= 1 
    
    # Ponemos a cero el último elemento
    if tidx == 0:
        aux[bidx] = shared_aux[2 * blockSize - 1]
        shared_aux[2 * blockSize - 1] = 0
        
    # Contrucción del árbol down-sweepe
    b = blockSize << 1
    d = 1
    while d < b:
        offset >>= 1
        cuda.syncthreads()
        
        if tidx < d:
            ai = offset * (2 * tidx + 1) - 1
            bi = offset * (2 * tidx + 2) - 1
            
            t = shared_aux[ai]
            shared_aux[ai] = shared_aux[bi]
            shared_aux[bi] += t
            
        d <<= 1
        
    cuda.syncthreads()
    
    # Guardamos los resultados obtenidos
    data_out[2 * idx] = shared_aux[2 * tidx]
    data_out[2 * idx + 1] = shared_aux[2 * tidx + 1]


@cuda.jit
def not_full_block_scan(data_in, data_out, aux, auxidx, elb, start_idx):
    # Índices y memoria compartida
    shared_aux = cuda.shared.array(0, numba.int32)

    tidx = cuda.threadIdx.x 
    blockSize =  cuda.blockDim.x

    # Cargamos en memoria compartida si procede, si no ponemos 0.
    a = 2 * tidx
    b = 2 * tidx + 1

    shared_aux[a] = data_in[start_idx + a] if a < elb else 0
    shared_aux[b] = data_in[start_idx + b] if b < elb else 0
    
    offset = 1

    d = blockSize
    while d > 0:
        cuda.syncthreads()
        
        if tidx < d:
            ai = offset * (2 * tidx + 1) - 1
            bi = offset * (2 * tidx + 2) - 1

            shared_aux[bi] += shared_aux[ai]
        offset <<= 1 
        d >>= 1

    # Poner último elemento a 0
    if tidx == 0:
        if auxidx != -1:
            aux[auxidx] = shared_aux[2 * blockSize - 1]

        shared_aux[2 * blockSize - 1] = 0
        
   
    d = 1
    c = blockSize << 1
    while d < c: 
        offset >>= 1
        cuda.syncthreads()
        
        if tidx < d:
            ai = offset * (2 * tidx + 1) - 1
            bi = offset * (2 * tidx + 2) - 1
            
            t = shared_aux[ai]
            shared_aux[ai] = shared_aux[bi]
            shared_aux[bi] += t
            
        d <<= 1
        
    cuda.syncthreads()
    
    # Escribimos salida
    if a < elb:
        data_out[start_idx + a] = shared_aux[a]
    if b < elb:
        data_out[start_idx + b] = shared_aux[b]

In [4]:
import numpy as np
x = np.array([2] * 6001)
d_x = cuda.to_device(x)
out = cuda.device_array(x.shape, np.int32)
scan(d_x, out)
out.copy_to_host()

array([    0,     2,     4, ..., 11996, 11998, 12000])